<a href="https://colab.research.google.com/github/panagiotismouts/machinelearning/blob/main/credit_risk_project/Part_2_Binning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binning
In this section we will be transforming the continuous variables into categorical, by dividing it randomly into a set of bins.

In [ ]:
#libraries
!pip install optbinning
!pip install python-pptx
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
from optbinning import OptimalBinning
import numpy as np
import os
from sklearn.model_selection import train_test_split
import pptx
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 7.8 MB/s eta 0:00:00
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470935 sha256=4bd89dcd34bfb2272fc3e683d3f04c1467f1d380afa45f8d5bd9ae608cc11a26
  Stored in directory: /root/.c

In [ ]:
#global variables setting
def globalvariables (text):
  if (text >= 1) or (text <=3):
    dataset_text = "Optimal Binning Dataset " + str(text)
    path = f"/content/drive/MyDrive/ergasia_ml/dataset_{text}_original.csv"
    return dataset_text, path

dataset_text, path = globalvariables(1) ## insert number of each dataset

In [ ]:
# load the dataset into a pandas dataframe
df = pd.read_csv(path)

# identify the data types of each column in the dataframe
dtypes = df.dtypes
print(df.shape)

# extract the names of the continuous variables
continuous_vars = []
for col in df.columns:
    if dtypes[col] == "float64" or dtypes[col] == "int64":
        continuous_vars.append(col)

# create a new dataframe containing only the continuous variables
df_continuous = df[continuous_vars]
print(df_continuous.shape)


(247032, 250)
(247032, 250)


# Exploratory Data Analysis
Describing the data.

In [ ]:
def describex(data):
    data = pd.DataFrame(data)
    stats = data.describe()
    skewness = data.skew()
    kurtosis = data.kurtosis()
    skewness_df = pd.DataFrame({'skewness':skewness}).T
    kurtosis_df = pd.DataFrame({'kurtosis':kurtosis}).T
    return stats.append([kurtosis_df,skewness_df])
  
describex(df_continuous)

<ipython-input-4-13216d5203fb>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return stats.append([kurtosis_df,skewness_df])


,SK_ID_CURR,day_to_actual_payment_count,max_days_of_delay,nummber_of_applications,Consumer loans,Cash loans,Revolving loans,XNA,Approved,Refused,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
count,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,...,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000,247032.000000
mean,278155.858618,27.743976,38.293254,4.998612,2.223696,2.217389,0.556406,0.001121,3.123413,0.876676,...,0.039218,0.004947,0.003991,0.007363,0.005611,0.005238,0.723720,0.210390,0.017714,0.007691
std,102862.182082,24.366741,29.478101,4.218016,1.844395,3.291494,1.020010,0.036477,2.147287,1.851184,...,0.194113,0.070159,0.063051,0.085494,0.074694,0.072186,0.447158,0.407586,0.131911,0.087362
min,100002.000000,0.000000,-25.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189000.500000,11.000000,23.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278094.500000,20.000000,32.000000,4.000000,2.000000,1.000000,0.000000,0.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,367318.250000,37.000000,44.000000,7.000000,3.000000,3.000000,1.000000,0.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,456255.000000,245.000000,3189.000000,73.000000,45.000000,60.000000,31.000000,3.000000,27.000000,68.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
kurtosis,-1.199739,4.952456,695.924342,10.469945,13.276616,16.270253,35.032820,1622.789093,3.828765,50.711919,...,20.540020,197.162832,245.548555,130.816577,173.242939,185.914772,-0.998730,0.019556,51.470654,125.027148
skewness,-0.000804,1.924136,11.141066,2.297892,2.412926,3.004493,3.953503,37.023710,1.566859,4.829083,...,4.747616,14.112450,15.733613,11.524562,13.237883,13.708146,-1.000639,1.421111,7.312335,11.270587


sum of int and float variables.

In [ ]:
ints = 0;
floats = 0;
for x in df_continuous.columns:
  if df_continuous[x].dtype == 'float64':
    floats+=1;
  else:
    ints+=1;

print(f'ints are:{ints}')
print(f'floats are:{floats}')

ints are:165
floats are:85


# Outlier detection.

In [ ]:
#excluding int64 values and any columns that have a data type other than float64.
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
df_cont_without_outliers = df_continuous.copy()

for x in df_cont_without_outliers.columns:
  if df_cont_without_outliers[x].dtype != "float64":
      df_cont_without_outliers.drop(x, axis=1, inplace=True) 

df_with_outliers = df_cont_without_outliers.copy()

descriptive_statistics = describex(df_with_outliers)

df_with_outliers.head()

<ipython-input-4-13216d5203fb>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return stats.append([kurtosis_df,skewness_df])


,max_days_of_delay,Consumer loans,Cash loans,Revolving loans,XNA,Approved,Refused,Canceled,Unused offer,max_previous_delayed_days,max_previous_anuity_to_credit,SK_DPD_DEF_count,SK_DPD_DEF_max,Closed,Active,Sold,Bad debt,max_amount_delayed,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,OWN_CAR_AGE,CNT_FAM_MEMBERS,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,percentage_of_credit_to_income,percentage_of_annuity_to_income
0,31.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-8.0,5.167002,0.0,0.0,6.0,2.0,0.0,0.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-3648.0,12.061091,1.0,0.083037,0.262949,0.139376,0.02470,0.036900,0.972200,0.619200,0.014300,0.000000,0.069000,0.083300,0.125000,0.036900,0.020200,0.019000,0.000000,0.000000,0.025200,0.038300,0.972200,0.634100,0.014400,0.00000,0.069000,0.083300,0.125000,0.037700,0.022000,0.019800,0.000000,0.000000,0.02500,0.036900,0.972200,0.624300,0.014400,0.000000,0.069000,0.083300,0.125000,0.037500,0.020500,0.019300,0.000000,0.000000,0.014900,2.0,2.0,2.0,2.0,-1134.0,0.0,0.0,0.0,0.0,0.0,1.0,200.788889,12.197778
1,14.0,2.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,-4.0,18.520000,0.0,0.0,3.0,1.0,0.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-1186.0,12.061091,2.0,0.311267,0.622246,0.510853,0.09590,0.052900,0.985100,0.796000,0.060500,0.080000,0.034500,0.291700,0.333300,0.013000,0.077300,0.054900,0.003900,0.009800,0.092400,0.053800,0.985100,0.804000,0.049700,0.08060,0.034500,0.291700,0.333300,0.012800,0.079000,0.055400,0.000000,0.000000,0.09680,0.052900,0.985100,0.798700,0.060800,0.080000,0.034500,0.291700,0.333300,0.013200,0.078700,0.055800,0.003900,0.010000,0.071400,1.0,0.0,1.0,0.0,-828.0,0.0,0.0,0.0,0.0,0.0,0.0,479.075000,13.221667
2,11.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-10.0,26.645031,0.0,0.0,2.0,0.0,0.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-4260.0,26.000000,1.0,0.502130,0.555912,0.729567,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.07449,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.11785,0.087955,0.977752,0.755746,0.044595,0.078078,0.149213,0.225897,0.231625,0.067169,0.101954,0.108607,0.008651,0.028236,0.102547,0.0,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0,200.000000,10.000000
3,31.0,2.0,4.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,12.549877,0.0,0.0,1.0,0.0,0.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-4311.0,12.061091,1.0,0.502130,0.322738,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.07449,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.11785,0.087955,0.977752,0.755746,0.044595,0.078078,0.149213,0.225897,0.231625,0.067169,0.101954,0.108607,0.008651,0.028236,0.102547,0.0,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0,422

Preproccessing - Finding the true integers.

In [ ]:
print(f"Before :{df_with_outliers.shape}")
# Iterate over each column
for column in df_with_outliers.columns:
    # Take 100 random samples from the column
    samples = df_with_outliers[column].sample(100, random_state=42)  # Adjust the random_state if needed
    
    unique_values = samples.unique()
    for num in unique_values:
      if isinstance(num, float) and (num - int(num)==0):
        print(column)
        # Convert the column to integer type
        df_with_outliers[column] = df_with_outliers[column].astype(int)

print(f"After :{df_with_outliers.shape}")
df_cont_without_outliers = df_with_outliers.copy()

Before :(247032, 85)
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
max_days_of_delay
Consumer loans
Consumer loans
Consumer loans
Consumer loans
Consumer loans
Consumer loans
Consumer loans
Consumer l

plotting the variables on boxplots.

In [ ]:
import zipfile
import os
sub_dfs = [df_cont_without_outliers.iloc[:, i:i+5] for i in range(0, df_cont_without_outliers.shape[1], 5)]

for i, sub_df in enumerate(sub_dfs):
    num_cols = sub_df.shape[1]
    fig, axes = plt.subplots(1, num_cols, figsize=(5*num_cols, 8))

    # Iterate over each column in the sub-dataframe
    for j, column in enumerate(sub_df.columns):
        ax = axes[j] if num_cols > 1 else axes
        sub_df.boxplot(column, ax=ax)
        ax.set_title(column)

    plt.suptitle(f"Boxplots for Sub-Dataframe {i+1}", y=1.05)
    plt.tight_layout()

    filename = f"boxplot_subdataframe_{i+1}.png"
    plt.savefig(filename, bbox_inches='tight')
    #plt.show() 
    plt.savefig(f"boxplot_subdataframe_{i+1}.png")
    plt.close()

with zipfile.ZipFile('boxplot_figures.zip', 'w') as zipf:
    for i in range(len(sub_dfs)):
        filename = f"boxplot_subdataframe_{i+1}.png"
        zipf.write(filename)
for i in range(len(sub_dfs)):
    filename = f"boxplot_subdataframe_{i+1}.png"
    os.remove(filename)

In [ ]:

#optimal binning den 8a kanw mono gia to train, 8a kanw kai gia to test alla
#kratwntas idia ta split
#8a kanw to idio gia ta x train kai ta x test

#ayto legetai stability gia na doume 
#population stability index

#8a krathsw to optimal binning pou vrhe gia to train
#kai 8a to efarmwsw sto test gia na dw an diathrh8hke stability gia to test.
# oi metavlht
#validate
#tuning
#vriskw tis statistika shmantikes metavlhtes

#meta 8a kanoume to shap, ekshgei ta apotelesmata tou xcboost.

Outliers code using the IQR method. It didn't work because our data have noice - decided to work with the noice.

In [ ]:
#outliers_df = df_cont_without_outliers.copy().iloc[0:0]
#df_with_outliers = df_cont_without_outliers.copy()

#for x in df_cont_without_outliers.columns:
#    q25,q75 = np.nanpercentile(df_cont_without_outliers.loc[:,x],[25,75])
#    intr_qr = q75-q25
 
#    max = q75+(1.5*intr_qr)
#    min = q25-(1.5*intr_qr)
#    outlier_mask = (df_cont_without_outliers.loc[:, x] > max) | (df_cont_without_outliers.loc[:, x] < min)
#    df_cont_without_outliers = df_cont_without_outliers[~outlier_mask]
  
    #df.loc[df[x] > max,x] = np.nan
#to find the dataframe that only has the outliers in it, we take the disjoint dataframe
#of the dataframe after the int64 clearence, and of the dataframe after the outlier clearance.

#print(f'Shape of original DataFrame: {df_continuous.shape}')
#print(f'Shape of DataFrame after int64 column clearance: {df_with_outliers.shape}')
#print(f'Shape of DataFrame after outlier clearance:{df_cont_without_outliers.shape}')

# Splitting the dataset into train - test - validate

In [ ]:
#the first dataset is called df.
from sklearn.model_selection import train_test_split
X = df.loc[:,df.columns.drop('TARGET')]
y = df.loc[:,'TARGET']

X_train, X_remainder, y_train, y_remainder = train_test_split(X, y, test_size=0.4, random_state=0)
X_test, X_validate, y_test, y_validate = train_test_split(X_remainder, y_remainder, test_size=0.5, random_state=0)

train_count = len(X_train)
test_count = len(X_test)
validate_count = len(X_validate)

total_count = len(X)
train_percent = (train_count / total_count) * 100
test_percent = (test_count / total_count) * 100
validate_percent = (validate_count / total_count) * 100

print(f"Train: {train_percent:.2f}% ({train_count} samples)")
print(f"Test: {test_percent:.2f}% ({test_count} samples)")
print(f"Validate: {validate_percent:.2f}% ({validate_count} samples)")


Train: 60.00% (148219 samples)
Test: 20.00% (49406 samples)
Validate: 20.00% (49407 samples)


# Binning

In [ ]:
!rm -rf figs

In [ ]:
import pandas as pd
from optbinning import ContinuousOptimalBinning, OptimalBinning
import os
import re

dict_tab = {}
dict_woe_plot = {}
dict_er_plot = {}
dict_qs = {}
dict_gini = {}
direc = "/content/figs" #for google colab, change to "./figs" to create locally.
os.makedirs(direc, exist_ok=True)

#fitting the binner ONLY using X_train
for column in X_train.columns:
    plot_path = ''

    binning = OptimalBinning(name=column, solver="cp", dtype="numerical", monotonic_trend="auto_asc_desc") #otan grapsw th diadikasia na checkarw pws diaxwrizei to binning me to solver : cp kai ti shmainei.
    #binning = ContinuousOptimalBinning(name=column, dtype="numerical", monotonic_trend="auto_asc_desc") #otan grapsw th diadikasia na checkarw pws diaxwrizei to binning me to solver : cp kai ti shmainei.
    binning.fit(X_train[column].values, y_train.values)
    #print(binning.status)

    binning_table = binning.binning_table #PPT
    tab = binning_table.build()#creating the binning table
    dict_tab.update({column:tab})#insert the table on the dictionary for X column

    #########################################################
    ####################    PLOTS     #######################
    if re.search("/", column):
        # Remove invalid characters from column name to use as filename
        plot_filename_woe = re.sub(r'[<>:"/\\|?*]', "", column) + '_WOE.png'
    else:
        plot_filename_woe = column + '_WOE.png'

    plot_path_woe = os.path.join(direc, plot_filename_woe)
    binning_table.plot(metric="woe", savefig=plot_path_woe) #show_bin_labels=True, to show bin labels.

    if re.search("/", column):
        # Remove invalid characters from column name to use as filename
        plot_filename_er = re.sub(r'[<>:"/\\|?*]', "", column) + '_ER.png'
    else:
        plot_filename_er = column + '_ER.png'

    plot_path_er = os.path.join(direc, plot_filename_er)

    binning_table.plot(metric="event_rate", savefig=plot_path_er)

    dict_woe_plot.update({column: plot_path_woe})
    dict_er_plot.update({column: plot_path_er})

    binning_table.analysis()
    dict_qs.update({column:round(binning_table.quality_score*1000, 2)})
    dict_gini.update({column:round(binning_table.gini, 5)})
    # qs: *100 kai meta integer.

#transformations
X_train_binned = pd.DataFrame()
for column in X_train.columns:
    binning = OptimalBinning(name=column, solver="cp", dtype="numerical", monotonic_trend="auto_asc_desc")
    binning.fit(X_train[column].values, y_train.values)
    X_train_binned[column] = binning.transform(X_train[column].values)

# Applying binning transformation on X_test
X_test_binned = pd.DataFrame()
for column in X_test.columns:
    binning = OptimalBinning(name=column, solver="cp", dtype="numerical", monotonic_trend="auto_asc_desc")
    binning.fit(X_train[column].values, y_train.values)
    X_test_binned[column] = binning.transform(X_test[column].values)

# Applying binning transformation on X_validate
X_val_binned = pd.DataFrame()
for column in X_validate.columns:
    binning = OptimalBinning(name=column, solver="cp", dtype="numerical", monotonic_trend="auto_asc_desc")
    binning.fit(X_train[column].values, y_train.values)
    X_val_binned[column] = binning.transform(X_validate[column].values)

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.

  Significance tests

    Bin A  Bin B  t-statistic      p-value      P[A > B]  P[B > A]
        0      1   149.946590 1.780877e-34  1.637047e-39       1.0
        1      2   253.040217 5.644957e-57 5.396557e-198       1.0

---------------------------------------------
OptimalBinning: Binary Binning Table Analysis
---------------------------------------------

  General metrics

    Gini index               0.09478906
    IV (Jeffrey)             0.04752516
    JS (Jensen-Shannon)      0.00589090
    Hellinger                0.00591564
    Triangular               0.02336904
    KS                       0.06380464
    HHI                      0.59520642
    HHI (normalized)         0.49400803
    Cramer's V               0.05815446
    Quality score            0.09946587

  Monotonic trend             ascending

  Significance tests

    Bin A  Bin B  t-statistic      p-value      P[A > B]  P[B > A]
        0      1   161.642650 4

<ipython-input-13-ae3e1cc9b249>:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_binned[column] = binning.transform(X_train[column].values)
<ipython-input-13-ae3e1cc9b249>:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train_binned[column] = binning.transform(X_train[column].values)
<ipython-input-13-ae3e1cc9b249>:61: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)
X_train_binned.head()

,SK_ID_CURR,day_to_actual_payment_count,max_days_of_delay,nummber_of_applications,Consumer loans,Cash loans,Revolving loans,XNA,Approved,Refused,Canceled,Unused offer,max_previous_delayed_days,max_previous_anuity_to_credit,SK_DPD_DEF_count,SK_DPD_DEF_max,Closed,Active,Sold,Bad debt,max_days_delayed,max_amount_delayed,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,percentage_of_credit_to_income,percentage_of_annuity_to_income,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANI

setting the presentation for the plots.

In [ ]:
import pptx
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import os

theme_path = "/content/drive/MyDrive/ergasia_ml/theme.pptx" ##change to presentation path.
presentation = Presentation(theme_path)

slides = presentation.slides

slide_layout = presentation.slide_layouts[6] 
slide = presentation.slides.add_slide(slide_layout)

slide_width = presentation.slide_width
slide_height = presentation.slide_height

left = (slide_width - Inches(4)) / 2
top = (slide_height - Inches(2)) / 2
width = Inches(4)
height = Inches(2)

textbox = slide.shapes.add_textbox(left, top, width, height)
text_frame = textbox.text_frame

p = text_frame.add_paragraph()
p.text = path
p.font.bold = True
p.font.size = Pt(36)
p.font.name = "Calibri"
p.font.color.rgb = RGBColor(81, 217, 241)
p.alignment = PP_ALIGN.CENTER

for key in dict_woe_plot.keys():
    # Create a new slide
    slide_layout = presentation.slide_layouts[6]  
    slide = presentation.slides.add_slide(slide_layout)

    #adding a header for each presentation.
    text_top = Inches(0.5)  # Adjust the top position as needed
    text_height = Inches(1)  # Adjust the height as needed
    text_box = slide.shapes.add_textbox(Inches(3.8), text_top, Inches(2), text_height)
    text_frame = text_box.text_frame

    p = text_frame.add_paragraph()
    p.text = key
    p.font.bold = True
    p.font.size = Pt(25)
    p.font.name = "Calibri"
    p.font.color.rgb = RGBColor(81, 217, 241)
    p.alignment = PP_ALIGN.CENTER

    # Add the WOE plot 
    woe_plot_path = dict_woe_plot[key]
    left = Inches(0.2)
    top = Inches(1.5) #OPTIMAL
    width = Inches(3.7) #OPTIMAL
    height = Inches(2.5) #OPTIMAL
    slide.shapes.add_picture(woe_plot_path, left, top, width, height)

    # Add the Event Rate plot 
    er_plot_path = dict_er_plot[key]
    left = Inches(0.2)
    top = Inches(4) #OPTIMAL
    width = Inches(3.7) #OPTIMAL
    height = Inches(2.5) #OPTIMAL
    slide.shapes.add_picture(er_plot_path, left, top, width, height)

    

    # Add the binning table to the slide
    table = dict_tab[key]
    num_rows, num_cols = table.shape
    left = Inches(4) #OPTIMAL
    top = Inches(1.5)
    width = Inches(5.9)
    height = Inches(0.0000001 * num_rows)
    table_placeholder = slide.shapes.add_table(num_rows + 1, num_cols, left, top, width, height).table

    # Set the column headers
    for i, header in enumerate(table.columns):
        cell = table_placeholder.cell(0, i)
        cell.text = header
        #cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
        cell.text_frame.paragraphs[0].font.size = Pt(10)

    # Set the table data
    for i in range(num_rows):
        for j in range(num_cols):
            cell = table_placeholder.cell(i + 1, j)
            cell.text = str(table.iloc[i, j])
            #cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
            cell.text_frame.paragraphs[0].font.size = Pt(6)
      
    #adding the information value.
    mtleft = Inches(0.6)
    mttop = Inches(6.6)
    mtwidth = Inches(3.3)
    mtheight = Inches(0.5)

    # Add a table shape to the slide
    mtable = slide.shapes.add_table(2, 1, mtleft, mttop, mtwidth, mtheight).table

    # Set the text for the first cell (dict_qs[key])
    mtable.cell(0, 0).text = "Quality Score: " + str(dict_qs[key])
    mtable.cell(0, 0).text_frame.paragraphs[0].font.size = Pt(12)

    # Set the text for the second cell (dict_gini[key])
    mtable.cell(1, 0).text ="Gini Coef: " + str(dict_gini[key])
    mtable.cell(1, 0).text_frame.paragraphs[0].font.size = Pt(12)

# Save the PowerPoint presentation
presentation.save("binning_plots.pptx")


In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)
X_train_binned["Refused"].value_counts()

 0.172744    95553
-0.080871    24245
-0.495034    16473
-0.246449    11948
Name: Refused, dtype: int64

# Model calibration - feature selection
Now that the binning is complete, we need to extract the information value the dataset variables offer.

1) feature importance using information gain.

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

def select_features_lasso(X_train, y_train):
    selected_features = {}
    for i in range(10,21):
        lasso_model = Lasso(alpha=0.1)
        sfs = SequentialFeatureSelector(estimator=lasso_model, scoring='roc_auc', n_features_to_select=i)
        sfs.fit(X_train, y_train)
        selected_features.update({i:X_train.columns[sfs.get_support()]})

    return selected_features

def select_features_ridge(X_train, y_train):
    
    selected_features_ridge = {}
    for i in range(10,21):
        ridge_model = Ridge(alpha=0.1)
        sfs = SequentialFeatureSelector(estimator=ridge_model, scoring='roc_auc', n_features_to_select=i)
        sfs.fit(X_train, y_train)
        selected_features_ridge.update({i:X_train.columns[sfs.get_support()]})

    return selected_features_ridge

def select_features_elasticnet(X_train, y_train):
    selected_features_elasticnet = {}
    for i in range(10,21):
        elasticnet_model = ElasticNet(alpha=0.1, l1_ratio=0.5)
        sfs = SequentialFeatureSelector(estimator=elasticnet_model, scoring='roc_auc', n_features_to_select=i)
        sfs.fit(X_train, y_train)
        selected_features_elasticnet.update({i:X_train.columns[sfs.get_support()]})

    return selected_features_elasticnet

def select_features_logistic(X_train, y_train):
    selected_features_logistic = {}
    for i in range(10,21):
        logistic_model = LogisticRegression()
        sfs = SequentialFeatureSelector(estimator=logistic_model, scoring='roc_auc', n_features_to_select=i)
        sfs.fit(X_train, y_train)
        selected_features_logistic.update({i:X_train.columns[sfs.get_support()]})

    return selected_features_logistic

# Assuming you have X_train_binned and y_train as your training data

# Step 1: Select features using Lasso
selected_features_lasso = select_features_lasso(X_train_binned, y_train)
print("Selected Features (Lasso):")
print(selected_features_lasso)

# Step 2: Select features using Ridge
selected_features_ridge = select_features_ridge(X_train_binned, y_train)
print("Selected Features (Ridge):")
print(selected_features_ridge)


# Step 3: Select features using ElasticNet
selected_features_elasticnet = select_features_elasticnet(X_train_binned, y_train)
print("Selected Features (ElasticNet):")
print(selected_features_elasticnet)


# Step 4: Select features using Logistic Regression
selected_features_logistic= select_features_logistic(X_train_binned, y_train)
print("Selected Features (Logistic Regression):")
print(selected_features_logistic)


excluding coefs that are 0.

In [ ]:
print(len(lasso_coefs))
print(len(elastic_coefs))
matching_indexes = elastic_coefs == lasso_coefs
matching_values = elastic_coefs[matching_indexes]
#print("Matching Values:")
#print(matching_values)

# Exclude matching values from lasso_coefs and elastic_coefs
lasso_coefs = lasso_coefs[~lasso_coefs.index.isin(matching_values.index)]
elastic_coefs = elastic_coefs[~elastic_coefs.index.isin(matching_values.index)]


# Sort the absolute coefficients
lasso_coefs = lasso_coefs.abs().sort_values(ascending=False)
elastic_coefs = elastic_coefs.abs().sort_values(ascending=False)

print("Lasso Coefficients:")
#print(lasso_coefs)
print(len(lasso_coefs))

print("Elastic Coefficients:")
#print(elastic_coefs)
print(len(elastic_coefs))


In [ ]:
    #
    #8a prospa8hsw to manual meta to validation.
    #
    #prepei na kanw feature importance kai sharp se metavlhtes.
    #prepei na epekshghsw giati oi metavlhtes einai kales.
    #8a grafw 2 kommatia
    #empyrical results kai methodology.
    #
#o kanw na ginetai kai pdf


#MODELING - trainning kai feature selection 8a epileksoume tis pio dynates metavlhtes.logistic (aplo, elastic, lasso kai ridge) 8Adialeksw metavlhtes.
#xrhsimopoiw sequential algor8imo stepwise kai rfecv h an vrw kalyterous.(chatgpt)

#OI LOGISTIC kai decision tree bgazoun feature importance. na plottarw kai to tree

#8elw ton best syndiasmo metavlhtwn pou megistopoiei to gini.
#sto telos, h px lasso me aytes tid parametrous, h stepwise me aytes tis metavlhtes. vasikes parametroi na mhn fortwnetai poly alla na mhn teleiwnei kai grhgora.

#den 8a yparxei montelo p 8a exei katw apo 5 metavlhtes. 8a apofasisw mesw mias metrikhs, gini coef to gini oso pio megalo einai toso pio kalo einai
#to montelo. 8a exoume koinh metrikh.
#to stacking de 8elei metrikh.

#PRIN PAW STO VALIDATION, print ta coef sto train
#kai tis dyo me8odologies sto teleytaio ma8hma na dokimasw na ta efarmwsw.

#GIA TO TELOS
#an ginetai na kanw append sto powerpoint, apo to ka8e montelo dialeksame ayta ta feautres kai to feature importance einai ayto,
#kai ta gini ka8e montelou prin kai meta.
#oti exei woe panw apo 1,3 einai ayto to binning.

#exoume kanei hdh separation
#to binning : risk variable. diaxwrhsame tous pelates se kalous kai kakous me vash thn ka8e metavlhth.
#isws na mhn kanoume plot to dentro alla 8a kanuome plot to feature importance.
#den 8a yparcxei plot p na mhn exoume kanei plot to feature importance.

#methodology, pros kai cons twn montelwn.
#methodology kai st ocost matrix. pws leitourgoun oloi oi algor8imoi 8ewrhtika.

#methodos gia black box, shap

#meta to validation, ksana 8a exw arxikopoihsei to gini me thn timh pou eixe prin (se ena dataframe.)

#TUNING
#sto validate, to logistic regration, ena grid search h ena random search me diaforetikes parametrous na dw an to gini ginetai kalytero, an ginetai kalyter oxrhsimopoiw to neo montelo
#8a m dinei ena gini coef, 
#skopos mas einai na anevei to gini.
#px paw sto decision tree, na xrhsimopousei ena learning rate allo

#molis teleiwsw m ayto
#cost matrix

#afou vroume ta features, koumpwnw to validation.

#to prunning na mhn bgazei katw apo 3, to depth sta decision trees.

#xrhsimopoiw 2 white box
#decision tree kai logistic (aplo, elastic, lasso kai ridge)
#prunning kai yper parametrous pou na kanei block, max depth = 20, minimum 2,3 gia decision tree (ensemble)
#8eloume 5 modela kai stacking.

#oste otan ginei to voting meta na mporei na apofasizei.

#sto stacking 8a xrhsimopoihsoume mono 3, nb(pi8anothtes), xgboost logistic regression(to continuous kommati).

#sta idia kyvika random forest.

# kai 3 blak box methods
#oi epekshgiseis vasizontai sto ma8hma kai se ayta p kanoume mazi sto methodology

#5 kladia decision tree, oxi random forest
# 2 methodologies random forest, bagging kai mia allh.